In [3]:
# python
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from storage_utils.patients_data_new import patients
import os
from biosppy.signals import ecg
import matplotlylib as pltly

/home/franciscosargo/.virtualenvs/pre-epi-seizures/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
# Open data file
# load the file

disk='/mnt/Seagate/'
files_dir = 'h5_files_backup/processing_datasets/'
file_name = 'seizure_datasets_new.h5'
file_path = disk + files_dir + file_name
print file_path
f = h5py.File(file_path, 'a')


/mnt/Seagate/h5_files_backup/processing_datasets/seizure_datasets_new.h5


NameError: name 'h5py' is not defined

In [2]:
# Compute Bipolar ECG, from available data

# patient parameters
patient_number=5

patient_info = patients[str(patient_number)]
patient_info

# feature for analysis
feature_name=('signals/3000_1200'
              '/raw'
              '/baseline_removal__filt:MedianFIR')
feature_name_for_struct = 'baseline_removal__filt:MedianFIR'

seizures = f[feature_name].keys()

# get seizures from patient
seizure_from_patient = [seizure
                         for seizure in seizures
                         if seizure.split('_')[0] == str(patient_number)]
print seizure_from_patient
    

NameError: name 'patients' is not defined

In [4]:
# Get the time parameters of the signal
Fs=1000
sec_before_onset = 50 * 60
sec_after_onset = 20 * 60
relative_time_seconds = np.linspace(-sec_before_onset, sec_after_onset, 
                           (sec_before_onset + sec_after_onset) * Fs)   # relative time-frame to EEG onset
time_sec = np.linspace(0, (sec_before_onset + sec_after_onset),
                           (sec_before_onset + sec_after_onset) * Fs)   # time-frame relative
                                                                        # to the beggining of acquisition

In [5]:
# Change Data in disk, correct for bipolarity of ECG
channels_from_patient = seizure_from_patient

channels_from_patient

hsm_convention_negative_pole_1 = 'ECG-'

hsm_convention_positive_pole_1 = 'Ecg'

hsm_convention_positive_pole_2 = 'ECG+'

positive_channels_name = [channel_name 
                     for channel_name in channels_from_patient
                     if hsm_convention_positive_pole_1 in channel_name
                     or hsm_convention_positive_pole_2 in channel_name]

negative_channels_name = [channel_name 
                     for channel_name in channels_from_patient
                     if hsm_convention_negative_pole_1 in channel_name]
window = [200, 300]

for negative_channel_name, positive_channel_name in zip(negative_channels_name,
                                              positive_channels_name):
    
    # load h5 datasets
    negative_channel_h5_dataset = f[feature_name + '/' + negative_channel_name]
    positive_channel_h5_dataset = f[feature_name + '/' + positive_channel_name]
    
    print negative_channel_h5_dataset
    print positive_channel_h5_dataset
    
    # load numpy arrays
    negative_channel_array = negative_channel_h5_dataset[:][0]
    positive_channel_array = positive_channel_h5_dataset[:][0]
    
    # Compute bipolar ECG feature
    bipolar_channel_array = (negative_channel_array - positive_channel_array)
    
    # Create name for new bipolar ECG channel
    bipolar_channel_name = negative_channel_name.replace('_'
                                                        + hsm_convention_negative_pole_1,
                                                        '')
    print bipolar_channel_name
    
    # Structure data into predefined pipeline structure
    feature_array = np.asarray([bipolar_channel_array])
    
    # Save data
    del f[feature_name + '/' + bipolar_channel_name]
    dset = f.create_dataset(feature_name + '/' + bipolar_channel_name,
                            data=feature_array)
    del f[feature_name + '/' + 'time_domain_' + bipolar_channel_name]
    time_dset = f.create_dataset(feature_name + '/' + 'time_domain_' + bipolar_channel_name, 
                                 data=time_sec)
    
    # add metadata
    dset.attrs['feature_legend'] = feature_name_for_struct
    time_dset.attrs['feature_legend'] = feature_name_for_struct
    

f.close()
print 'Files saved!'
    

<HDF5 dataset "5_FA7773TC_ECG-_2016-08-23 08:24:24.000000_0": shape (1, 4200000), type "<f8">
<HDF5 dataset "5_FA7773TC_Ecg_2016-08-23 08:24:24.000000_0": shape (1, 4200000), type "<f8">
5_FA7773TC_2016-08-23 08:24:24.000000_0
<HDF5 dataset "5_FA7773TT_ECG-_2016-08-24 13:48:00.000000_1": shape (1, 4200000), type "<f8">
<HDF5 dataset "5_FA7773TT_Ecg_2016-08-24 13:48:00.000000_1": shape (1, 4200000), type "<f8">
5_FA7773TT_2016-08-24 13:48:00.000000_1
<HDF5 dataset "5_FA7773U1_ECG-_2016-08-25 01:44:21.000000_2": shape (1, 4200000), type "<f8">
<HDF5 dataset "5_FA7773U1_Ecg_2016-08-25 01:44:21.000000_2": shape (1, 4200000), type "<f8">
5_FA7773U1_2016-08-25 01:44:21.000000_2
<HDF5 dataset "5_FA7773U3_ECG-_2016-08-25 05:44:46.000000_3": shape (1, 4200000), type "<f8">
<HDF5 dataset "5_FA7773U3_Ecg_2016-08-25 05:44:46.000000_3": shape (1, 4200000), type "<f8">
5_FA7773U3_2016-08-25 05:44:46.000000_3
<HDF5 dataset "5_FA7773U4_ECG-_2016-08-25 07:44:59.000000_4": shape (1, 4200000), type "<f8"